# LingMess: Linguistically Informed Multi Expert Scorers for Coreference Resolution
This notebook is the official Demo of the paper ["LingMess: Linguistically Informed Multi Expert Scorers for Coreference Resolution"](https://arxiv.org/abs/2205.12644).


# Clone the code repository

In [ ]:
!git clone https://github.com/shon-otmazgin/lingmess-coref.git

Cloning into 'lingmess-coref'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 396 (delta 98), reused 102 (delta 47), pack-reused 238
Receiving objects: 100% (396/396), 97.83 KiB | 7.52 MiB/s, done.
Resolving deltas: 100% (251/251), done.


# Install requirements

In [ ]:
!python --version # make sure to use python 3.7, otherwise install it.

Python 3.7.13


In [ ]:
cd lingmess-coref/

/content/lingmess-coref


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.7 MB 15.2 MB/s 
     |████████████████████████████████| 38.1 MB 367 kB/s 
     |████████████████████████████████| 12.8 MB 31.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3a19e000 @  0x7fc32b989615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████████████████████████████| 1.2 MB 62.0 MB/s 
     |████████████████████████████████| 2.9 MB 53.8 MB/s 
     |████████████████████████████████| 325 kB 73.2 MB/s 
     |████████████████████████████████| 1.8 MB 48.7 MB/s 
     |███████████████████

There are some incompatible ^^^ but they for packages we not using so we can continue.

# Prepare your custom data for Inference

In [ ]:
import json
import uuid

examples = [
        {'doc_key': uuid.uuid4().hex, 'text': 'The man tried to put the boot on his foot but it was too small.'},
        {'doc_key': uuid.uuid4().hex, 'text': 'Some apologizing was needed in the relationship after the argument because it is soothing.'}
    ]
with open('toy_data_raw.jsonlines', 'w') as f:
    for doc in examples:
        f.write(json.dumps(doc) + "\n")

# Inference

In [ ]:
!python run.py \
        --output_file=predictions.jsonlines \
        --model_name_or_path=biu-nlp/lingmess-coref \
        --test_file=toy_data_raw.jsonlines \
        --eval_split=test \
        --max_tokens_in_batch=15000 \
        --device=cuda:0

06/15/2022 16:13:16 - INFO - __main__ -   missing_keys: []
06/15/2022 16:13:16 - INFO - __main__ -   unexpected_keys: []
06/15/2022 16:13:16 - INFO - __main__ -   mismatched_keys: []
06/15/2022 16:13:16 - INFO - __main__ -   error_msgs: []
06/15/2022 16:13:16 - INFO - __main__ -   Parameters: 590.0M, Transformer: 434.6M, Head: 155.4M
06/15/2022 16:13:16 - INFO - coref_dataset -   Creating dataset for {'train': None, 'dev': None, 'test': 'toy_data_raw.jsonlines'}
06/15/2022 16:13:18 - INFO - util -   Tokenize documents using Spacy...
100% 2/2 [00:00<00:00, 81.54it/s]
06/15/2022 16:13:18 - INFO - coref_dataset -   Tokenize documents...
06/15/2022 16:13:18 - WARNING - datasets.fingerprint -   Parameter 'function'=<function encode at 0x7fc4bd45acb0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to

# Resolve the clusters

In [ ]:
import json

predicted_docs = []
with open('predictions.jsonlines', 'r') as f:
    for line in f:
        predicted_docs.append(json.loads(line))

for i, doc in enumerate(predicted_docs):
    tokens = doc['tokens']
    predicted_clusters = doc['clusters']
    doc_key = doc['doc_key']
    print(f"##### doc_key: {doc_key}")

    for cluster in predicted_clusters:
        text_cluster = []
        for start, end in cluster:
            text_cluster.append(tokens[start:end+1])
        print(text_cluster)
    print("###########################")
    print()

##### doc_key: 300b22c2baa447aea33e34b2a0bcbfe3
[['The', 'man'], ['his']]
[['the', 'boot'], ['it']]
###########################

##### doc_key: 022810dea7c04365aff96fed73f3dd23
[['Some', 'apologizing'], ['it']]
###########################

